# To do

- Deal with under determined problems
- Integrate mixed-integer programming for certain foods that are discrete: eggs, bread, etc
- Convert from grams to more appropriate measure: one egg instead of 50g of eggs, etc
- Move food definition into a csv file, how would I instanciate them?
- Weigh protein by protein quality, e.g. chicken protein would be better than rice protein

In [12]:
from nutrition import Food, get_macros_per_meal, display_goal, MealSolver, display_meal

In [13]:
rice = Food("Rice", "grams", serving_size=47, cals =170, carbs=37, proteins=3, fats=0)
vegetables = Food("Vegetables", "grams", serving_size=100, cals=28, carbs=3, proteins=3, fats=0.5)
chicken = Food("Chicken breast", "grams", serving_size=112, cals=100, carbs=0, proteins=24, fats=0.5 )
oil = Food("Canola oil", "grams", serving_size=14, cals=120, carbs =0, proteins=0,fats=14)
ground_beef_85 = Food("85% ground beef", "grams", serving_size=113.4, cals = 240, carbs = 0, proteins=21, fats=17)
eggs = Food("Eggs", "grams", serving_size=50, cals =60, carbs=0, proteins=6, fats=4)
egg_whites = Food("Egg whites", "grams", serving_size=46, cals=25, carbs=0, proteins=5, fats=0)
whole_milk = Food("whole milk", "ml", serving_size=240, cals=150, carbs=12, proteins=8, fats=8 )
one_percent_millk = Food("1% milk", "ml", serving_size=240, cals=110, carbs=13, proteins=8, fats=2.5)
bread = Food( "Bread", "grams", serving_size=28, cals=70, carbs=14, proteins=3, fats=1 )

print(rice)
print(vegetables)
print(chicken)
print(oil)
print(ground_beef_85)
print(eggs)
print(egg_whites)
print(whole_milk)
print(one_percent_millk)
print(bread)

Name:                 Rice, cals: 3.62/g, carbs: 0.79/g, proteins: 0.06/g, fats 0.00/g
Name:           Vegetables, cals: 0.28/g, carbs: 0.03/g, proteins: 0.03/g, fats 0.01/g
Name:       Chicken breast, cals: 0.89/g, carbs: 0.00/g, proteins: 0.21/g, fats 0.00/g
Name:           Canola oil, cals: 8.57/g, carbs: 0.00/g, proteins: 0.00/g, fats 1.00/g
Name:      85% ground beef, cals: 2.12/g, carbs: 0.00/g, proteins: 0.19/g, fats 0.15/g
Name:                 Eggs, cals: 1.20/g, carbs: 0.00/g, proteins: 0.12/g, fats 0.08/g
Name:           Egg whites, cals: 0.54/g, carbs: 0.00/g, proteins: 0.11/g, fats 0.00/g
Name:           whole milk, cals: 0.62/g, carbs: 0.05/g, proteins: 0.03/g, fats 0.03/g
Name:              1% milk, cals: 0.46/g, carbs: 0.05/g, proteins: 0.03/g, fats 0.01/g
Name:                Bread, cals: 2.50/g, carbs: 0.50/g, proteins: 0.11/g, fats 0.04/g


In [14]:
#inputs
bodyweight = 150
n_meals = 4
protein_ratio = 1
fat_ratio = 0.4
daily_cals = 2200

cals_per_meal, protein_per_meal, fats_per_meal = get_macros_per_meal(bodyweight, protein_ratio, fat_ratio, daily_cals, n_meals)
display_goal(cals_per_meal, protein_per_meal, fats_per_meal)

Goal:
   cals: 550, proteins: 38g, fats 15g


In [15]:
foods = [rice, ground_beef_85, vegetables]

solver = MealSolver(foods)
solver.add_total_constraint("proteins",protein_per_meal)
solver.add_total_constraint("cals", cals_per_meal)
solver.add_total_constraint("fats", fats_per_meal)
solver.add_quant_constraint(vegetables, 100)

x, res = solver.solve()

display_goal(cals_per_meal, protein_per_meal, fats_per_meal)
print()
display_meal(foods, x)

Goal:
   cals: 550, proteins: 38g, fats 15g

Meal:
   -66g of Rice 
   -133g of 85% ground beef 
   -100g of Vegetables 
Macros:
   cals: 550, carbs: 55g, proteins: 32g, fats 20g


Breakfast

In [16]:
print("Current meal")
display_meal([eggs, egg_whites, whole_milk],[150, 46*2,1.5*240])

Current meal
Meal:
   -150g of Eggs 
   -92g of Egg whites 
   -360g of whole milk 
Macros:
   cals: 455, carbs: 18g, proteins: 40g, fats 24g


In [27]:
#underdetermined problem

foods = [eggs, egg_whites, one_percent_millk, bread]

solver = MealSolver(foods)
solver.add_total_constraint("proteins",protein_per_meal)
solver.add_total_constraint("cals", cals_per_meal)
solver.add_total_constraint("fats", fats_per_meal)

x, res = solver.solve()
display_meal(foods, x)

Meal:
   -122g of Eggs 
   -66g of Egg whites 
   -0g of 1% milk 
   -147g of Bread 
Macros:
   cals: 550, carbs: 74g, proteins: 38g, fats 15g
